In [1]:
# Transformers installation
! pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 57.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 14.6 MB/s eta 0:00:00


## Loading Data

In [2]:
from datasets import load_dataset, Dataset
import pandas as pd

df_train= pd.read_csv('/content/train.csv')
df_dev= pd.read_csv('/content/dev.csv')

df_train.rename(columns= {'initial_request': 'text','clarification_need': 'label'}, inplace= True)
df_dev.rename(columns= {'initial_request': 'text','clarification_need': 'label'}, inplace= True)

df_train['label']= [df_train['label'][i]-1 for i in range(len(df_train['label']))]  #since model uses labels 0-3
df_dev['label']= [df_dev['label'][i]-1 for i in range(len(df_dev['label']))]


train_data = Dataset.from_pandas(df_train) #.select(range(184))
dev_data = Dataset.from_pandas(df_dev) #.select(range(48))

In [3]:
train_data[0]

{'text': 'Tell me about Obama family tree.', 'label': 1}

## Tokenizing Data

In [4]:
from transformers import AutoTokenizer, OpenAIGPTForSequenceClassification

from transformers import AutoModelForSequenceClassification, AutoModel
from transformers import ElectraTokenizer, ElectraForSequenceClassification

tokenizer = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')
model = ElectraForSequenceClassification.from_pretrained('google/electra-small-discriminator', num_labels=4)

# if tokenizer.pad_token is None:
#     tokenizer.add_special_tokens({'pad_token': '[PAD]'})
#     model.resize_token_embeddings(len(tokenizer))

# model.config.pad_token_id = tokenizer.pad_token_id

# def tokenize_function(examples):

#     return tokenizer(examples["text"], padding="max_length", truncation=True) #padding="max_length", add this argument if needed

def tokenize_function(examples):

    tnizer = tokenizer(examples["text"], padding="max_length", truncation=True)
    # tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    # model.resize_token_embeddings(len(tokenizer))
    return tnizer #padding="max_length", add this argument if needed

tokenized_train = train_data.map(tokenize_function, batched=True)
tokenized_dev = dev_data.map(tokenize_function, batched=True)

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

Map:   0%|          | 0/187 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [5]:
tokenized_train = tokenized_train.remove_columns(["text"])
tokenized_train = tokenized_train.rename_column("label", "labels")
tokenized_train.set_format("torch")

tokenized_dev = tokenized_dev.remove_columns(["text"])
tokenized_dev = tokenized_dev.rename_column("label", "labels")
tokenized_dev.set_format("torch")

### Dataloader

In [6]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_train, batch_size=8)
eval_dataloader = DataLoader(tokenized_dev, batch_size=8)

### Model

In [7]:
import torch

In [8]:
# from transformers import AutoModelForSequenceClassification, AutoModel

# model = AutoModelForSequenceClassification.from_pretrained("t5", num_labels=4)

In [8]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [9]:
from transformers import get_scheduler

num_epochs = 20
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [10]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# device = torch.device("cpu")
model.to(device)

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (embeddings_project): Linear(in_features=128, out_features=256, bias=True)
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0): ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=256, out_features=256, bias=True)
              (key): Linear(in_features=256, out_features=256, bias=True)
              (value): Linear(in_features=256, out_features=256, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_

### Training

In [11]:
import os
from datasets import load_metric
from tqdm.auto import tqdm

In [12]:
def train_one_batch(model, batch, optimizer, lr_scheduler):
  batch = {k: v.to(device) for k, v in batch.items()}
  outputs = model(**batch)
  loss = outputs.loss
  loss.backward()

  optimizer.step()
  lr_scheduler.step()
  optimizer.zero_grad()

  return loss, model, optimizer, lr_scheduler


def train_one_epoch(model, train_dataloader, optimizer, lr_scheduler):
  model.train()
  loss = 0
  progress_bar = tqdm(range(len(train_dataloader)))

  for batch in train_dataloader:
    running_loss, model, optimizer, lr_scheduler = train_one_batch(model, batch, optimizer, lr_scheduler)
    loss += running_loss
    progress_bar.update(1)

  loss = loss / len(train_dataloader)
  return loss, model, optimizer, lr_scheduler
  

def get_val_loss(model, val_dataloader):
  model.eval()
  loss = 0

  metric = load_metric('f1')
  with torch.no_grad():
    for batch in val_dataloader:
      batch = {k: v.to(device) for k, v in batch.items()}
      outputs = model(**batch)
      running_loss = outputs.loss
      loss += running_loss

      logits = outputs.logits
      predictions = torch.argmax(logits, dim=-1)
      metric.add_batch(predictions=predictions, references=batch["labels"])
  
  f1_score = metric.compute(average= 'weighted')
  
  return loss / len(val_dataloader), f1_score

In [ ]:
# num_epochs=10
for epoch in range(num_epochs):
  print(f'Epoch: [{epoch+1} / {num_epochs}]:')
  t_loss, model, optimizer, lr_scheduler = train_one_epoch(model, train_dataloader, optimizer, lr_scheduler)
  v_loss, f1_score = get_val_loss(model, eval_dataloader)
  # print
  print(f"\tLoss -> Train: {t_loss:.5f} | Val: {v_loss} | F1 Score: {f1_score['f1']}")
  # save
  # DST Folder
  DST = '/content/checkpoints'
  if not os.path.isdir(DST):
    os.makedirs(DST)
  path = os.path.join(DST, f'epoch_{epoch+1}.pth')
  torch.save(model.state_dict(), path)

Epoch: [1 / 20]:


  0%|          | 0/24 [00:00<?, ?it/s]

### Testing

In [39]:
df_test= pd.read_csv('/content/test_with_labels.csv')

df_test.rename(columns= {'initial_request': 'text','clarification_need': 'label'}, inplace= True)
df_test['label']= [df_test['label'][i]-1 for i in range(len(df_test['label']))]
test_data = Dataset.from_pandas(df_test) #.select(range(184))
tokenized_test = test_data.map(tokenize_function, batched=True)

tokenized_test = tokenized_test.remove_columns(["text"])
tokenized_test = tokenized_test.rename_column("label", "labels")
tokenized_test.set_format("torch")
test_dataloader = DataLoader(tokenized_test, batch_size=8)

# model = TheModelClass(*args, **kwargs)
modelnew = ElectraForSequenceClassification.from_pretrained('google/electra-small-discriminator', num_labels=4)
modelnew.to(device)
modelnew.load_state_dict(torch.load('/content/checkpoints/epoch_1.pth'))
modelnew.eval()
# modelnew= torch.load('/content/checkpoints/epoch_8.pth',weights_only= True)

test_loss, f1_score = get_val_loss(modelnew, test_dataloader)

print(f"Test loss is: {test_loss:.5f}, F1 score is: {f1_score['f1']:.5f}")

Map:   0%|          | 0/61 [00:00<?, ? examples/s]

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

Test loss is: 1.27981, F1 score is: 0.33507
